In [2]:
import numpy as np
import pygame
from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)
pygame.init()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [25]:
class Node(pygame.sprite.Sprite):

    COLOR_INIT = (100,100,100)
    COLOR_CONNECTED = (0,255,0)
    WHITE = (255,255,255)
    DIST = 20
    Kp = 0.1
    Kd = Kp/3.
    BUFFER_SIZE = 20
    R=10

    def __init__(self, x=0, y=0,
                 coef=0.9):
        super().__init__()
        self.surf = pygame.Surface([Node.R*2, Node.R*2])
        self.surf.fill(Node.WHITE)
        self.surf.set_colorkey(Node.WHITE)
        pygame.draw.circle(surface=self.surf,
            color=self.COLOR_INIT,center=(Node.R,Node.R),
            radius=Node.R)
        
        self.rect = self.surf.get_rect()
        self.rect.x = x-Node.R
        self.rect.y = y-Node.R
        self.past_error = 0.0
        self.next = None
        self.buffer = []
        self.target_x = None
        self.target_y = None
        self.history_coef = coef

    @property
    def center(self):
        return self.rect.x+Node.R,self.rect.y+Node.R

    def update(self,x,y):
        cx,cy = self.center

        if self.next is not None:
            self.next.update(cx, cy)

        if self.target_x is None:
            self.target_x = x
            self.target_y = y
        else:
            self.target_x = self.history_coef*self.target_x + \
                (1-self.history_coef)*x
            self.target_y = self.history_coef*self.target_y + \
                (1-self.history_coef)*y
        x = self.target_x
        y = self.target_y
        
        d = np.sqrt((cx-x)**2 + (cy-y)**2)
        e = np.abs(d-self.DIST)
        de = e-self.past_error
        self.past_error = e
        e_ = (e*Node.Kp + de*Node.Kd)
        dx = (x-cx)/d*e_
        dy = (y-cy)/d*e_
        self.rect.x += dx
        self.rect.y += dy

class Snake():
    def __init__(self,init_node,
                 capture_dist=80) -> None:
        self.nodes = pygame.sprite.Group()
        self.nodes.add(init_node)
        self.head = init_node
        self.head_traj = np.array([[init_node.center]])
        self.traj_cum_len = np.array([0.])
        self.capture_dist = capture_dist
        self.seg_len = 20 # length of each segment
        self.n_nodes = 1

    def update(self,x,y, skip_head=False):
        # update head trajectory
        cx,cy = self.head.center
        px,py = self.head_traj[-1]
        self.head_traj = np.vstack((self.head_traj,[cx,cy]))
        d = np.sqrt((cx-px)**2 + (cy-py)**2)
        self.traj_cum_len += d
        self.traj_cum_len = np.concatenate((self.traj_cum_len,0.0))

        if self.traj_cum_len[0] > self.seg_len*(self.n_nodes+1):
            self.traj_cum_len.

        if not skip_head:
            self.head.update(x,y)
        else:
            cx, cy = self.head.center
            if self.head.next is not None:
                self.head.next.update(cx, cy)

    def append_head(self, new_node):
        new_node.next = self.head
        self.head = new_node
        self.nodes.add(new_node)
    
    def capture(self, node : Node):
        """check if head is within some threshold of the rogue
        node

        Args:
            node (_type_): _description_
        """
        cx1,cy1 = self.head.center
        cx2,cy2 = node.center
        d = np.sqrt((cx1-cx2)**2 + (cy1-cy2)**2)
        print(cx1, cy1, cx2, cy2,d)
        if d < self.capture_dist:
            node.next = self.head
            self.head = node
            self.nodes.add(node)
            print("captured")
            return 1
        return 0

In [26]:
# Set up the drawing window
WIDTH = 500
HEIGHT = 500

def spawn_new_node():
    return Node(x=np.random.randint(Node.R,WIDTH-Node.R),
            y=np.random.randint(Node.R,HEIGHT-Node.R))
screen = pygame.display.set_mode([WIDTH, HEIGHT])
screen.fill((255, 255, 255))
clock = pygame.time.Clock()

running = True
init_node = Node(100,100)
snake = Snake(init_node)
# node2 = Node(x=85,y=100)
# snake.append_head(node2)
all_nodes = pygame.sprite.Group()
all_nodes.add(init_node)
# spawn new node
new_node = spawn_new_node()
all_nodes.add(new_node)
# Main loop
spawn_counter = 1
capture_counter = 0
ignore_counts = 10
ignore_counter = 0
while running:
    # Look at every event in the queue
    for event in pygame.event.get():
        # Did the user hit a key?
        if event.type == KEYDOWN:
            # Was it the Escape key? If so  , stop the loop.
            if event.key == K_ESCAPE:
                running = False

        # Did the user click the window close button? If so, stop the loop.
        elif event.type == QUIT:
            running = False

    if snake.capture(new_node):
        capture_counter += 1
        ignore_counter = 1
    
    if ignore_counter > 0:
        ignore_counter += 1
        if ignore_counter > ignore_counts:
            ignore_counter = 0

    if capture_counter >= spawn_counter:
        new_node = spawn_new_node()
        all_nodes.add(new_node)
        spawn_counter += 1

    print(len(snake.nodes))
    x, y = pygame.mouse.get_pos()
    snake.update(x,y,skip_head=ignore_counter>0)
    
    screen.fill((255, 255, 255))
    for node in all_nodes:
        screen.blit(node.surf, node.rect)

    pygame.display.update()

    clock.tick(30)

pygame.quit()

100 100 305 246 251.67637950352037
1
89 89 305 246 267.02996086581743
1
82 82 305 246 276.81221071332817
1
75 75 305 246 286.60251220113196
1
69 69 305 246 295.0
1
64 64 305 246 302.00165562460086
1
59 59 305 246 309.00647242412253
1
55 55 305 246 314.61246002026047
1
51 51 305 246 320.22023671217283
1
47 47 305 246 325.8297101247828
1
44 44 305 246 330.037876614185
1
41 41 305 246 334.2469147202409
1
38 38 305 246 338.45679192475956
1
36 36 305 246 341.2638275586793
1
34 34 305 246 344.0712135590538
1
32 32 305 246 346.87894141904894
1
30 30 305 246 349.6870029040256
1
28 28 305 246 352.4953900407777
1
27 27 305 246 353.89970330589426
1
26 26 305 246 355.30409510727566
1
25 25 305 246 356.70856451731015
1
24 24 305 246 358.11311062288684
1
23 23 305 246 359.5177325251148
1
22 22 305 246 360.9224293390479
1
21 21 305 246 362.32720019341633
1
20 20 305 246 363.73204423036475
1
19 19 305 246 365.1369606051954
1
19 19 305 246 365.1369606051954
1
19 19 305 246 365.1369606051954
1
19 19 305

/tmp/ipykernel_75789/2143366012.py:58: RuntimeWarning: invalid value encountered in scalar divide
  dx = (x-cx)/d*e_
/tmp/ipykernel_75789/2143366012.py:59: RuntimeWarning: invalid value encountered in scalar divide
  dy = (y-cy)/d*e_


-1122758433 -1122758467 374 341 1587820732.8403432
2
-1014477184 -1014477222 374 341 1434687924.782272
2
-916638810 -916638851 374 341 1296323571.4722612
2
-828236178 -828236222 374 341 1171303372.469704
2
-748359284 -748359331 374 341 1058340387.7759861
2
-676185888 -676185938 374 341 956271794.4315823
2
-610973048 -610973101 374 341 864046913.784036
2
-552049474 -552049529 374 341 780716397.7039549
2
-498808615 -498808672 374 341 705422454.247975
2
-450702418 -450702477 374 341 637390019.4305967
2
-407235686 -407235746 374 341 575918778.2312669
2
-367960978 -367961039 374 341 520375954.2265306
2
-332474007 -332474069 374 341 470189799.2578968
2
-300409475 -300409538 374 341 424843703.93945754
2
-271437315 -271437379 374 341 383870883.04332113
2
-245259292 -245259356 374 341 346849567.88060635
2
-221605933 -221605998 374 341 313398667.49423355
2
-200233754 -200233820 374 341 283173842.802075
2
-180922754 -180922821 374 341 255863965.40619513
2
-163474148 -163474215 374 341 231187910.1

In [54]:
# Import and initialize the pygame library
import pygame
pygame.init()

# Set up the drawing window
screen = pygame.display.set_mode([500, 500])

# Run until the user asks to quit
running = True
while running:

    # Did the user click the window close button?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the background with white
    screen.fill((255, 255, 255))

    # Draw a solid blue circle in the center
    pygame.draw.circle(screen, (0, 0, 255), (250, 250), 75)

    # Flip the display
    pygame.display.flip()

# Done! Time to quit.
pygame.quit()